<a href="https://colab.research.google.com/github/adc257/info4940-sitcom/blob/main/10nContext_LLAMA_testing2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 7.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import random
import json
import os

In [3]:
!git clone https://github.com/adc257/info4940-sitcom.git

Cloning into 'info4940-sitcom'...
remote: Enumerating objects: 1116, done.
remote: Counting objects: 100% (1116/1116), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 1116 (delta 968), reused 1046 (delta 917), pack-reused 0
Receiving objects: 100% (1116/1116), 14.74 MiB | 20.50 MiB/s, done.
Resolving deltas: 100% (968/968), done.


# Defining some helper functions

In [4]:
def list_files(start_path):
    file_paths = []
    for root, dirs, files in os.walk(start_path):
        for file in files:
            file_paths.append(os.path.join(root, file))

    file_paths.sort()
    return file_paths

def fPathList_TO_DialogueWithContext(filePathList, nSentenceContext):
  humorous_inputs = []
  non_humorous_inputs = []

  # formatted_inputs = []
  # labels = []

  for episodePath in filePathList:

    with open(episodePath, 'r') as file:
      data = json.load(file)
      data = [info for (_, info) in data.items()]


      i = nSentenceContext
      while i < len(data):
        context = [data[j] for j in range(i-nSentenceContext, i)]
        target = data[i]

        if 'isHumor' in target:
          humorous_inputs.append(transformInput(context, target))
        else:
          non_humorous_inputs.append(transformInput(context, target))

        i += 1

  return humorous_inputs, non_humorous_inputs

def sampleEven(humorousData, nonHumorousData, sample_size_per_class, rand_seed):
  sample_size_per_class = min(sample_size_per_class, len(humorousData), len(nonHumorousData))

  sample_inputs = []
  sample_labels = []

  data = humorousData
  test_size = (len(data) - sample_size_per_class) / len(data)
  sample, _ = train_test_split(data, test_size=test_size, random_state=rand_seed)
  sample_inputs += sample
  sample_labels += [1]*len(sample)


  data = nonHumorousData
  test_size = (len(data) - sample_size_per_class) / len(data)
  sample, _ = train_test_split(data, test_size=test_size, random_state=rand_seed)
  sample_inputs += sample
  sample_labels += [0]*len(sample)

  return sample_inputs, sample_labels
def testLLAMA(inputs, labels, printResultsEveryN=50):
  resultsData = {
      "truePos": [],
      "trueNeg": [],
      "falsePos": [],
      "falseNeg": [],
  }

  results = {
      "truePos": 0,
      "trueNeg": 0,
      "falsePos": 0,
      "falseNeg": 0,
  }

  correct = 0
  total = 0
  pred = []

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  for i in range(len(inputs)):
    input = inputs[i]
    label = labels[i]

    input_ids = tokenizer.apply_chat_template(
      input,
      add_generation_prompt=True,
      return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
      input_ids,
      max_new_tokens=50,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.3,
      top_p=0.9,
      num_return_sequences=1,
    )

    response = outputs[0][input_ids.shape[-1]:]
    response = tokenizer.decode(response, skip_special_tokens=True)

    resultsDict = {
        "input": input,
        "output": response,
    }


    if str(response) == str(label):
      # true
      if str(label) == "1":
        # pos
        resultsData["truePos"].append(resultsDict)
        results["truePos"] += 1
      else:
        # neg
        resultsData["trueNeg"].append(resultsDict)
        results["trueNeg"] += 1
      correct += 1
    else:
      # false
      if str(label) == "1":
        # pos, pred neg
        resultsData["falseNeg"].append(resultsDict)
        results["falseNeg"] += 1
      else:
        # neg, pred pos
        resultsData["falsePos"].append(resultsDict)
        results["falsePos"] += 1
    pred.append(response)
    total += 1

    if i % printResultsEveryN == 0:
      print(f"Test: {i+1} of  {len(inputs)}")
      print(f"Success rate: {correct / total:.2%}\n")

      print(f"Results summary: {results}\n")


  print(f"Final success rate: {correct / total:.2%}")

  # true = [str(i) for i in labels]
  # report = classification_report(y_pred=pred, y_true=true, target_names=["Non-Humorous","Humorous"])
  report = None

  return results, resultsData, pred, report

## Defining input formatter

In [5]:
def transformInput(context, target):
  messages = []

  separator = "\n"
  quotation_token = '"'

  # formatted_input = ""

  systemPrmpt = {
      'role': 'system',
      'content': "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."
  }
  # append systemPrmpt
  messages.append(systemPrmpt)

  systemPrmpt2 = {
      'role': 'system',
      'content': ""
  }
  userPrmpt = {
      'role': 'user',
      'content': ""
  }
  userPrmpt2 = {
      'role': 'user',
      'content': ""
  }
  asstPrmpt = {
      'role': 'assistant',
      'content': ""
  }

  current_scene = context[0]['Scene']

  userPrmpt['content'] += "Is the following target_moment from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond '1' for humorous or '0' for not humorous:"
  userPrmpt['content'] += separator*2
  entry = target
  if entry['Scene'] != current_scene:
    current_scene = entry["Scene"]
    userPrmpt['content'] += "The scene changes to: "
  else:
    userPrmpt['content'] += "The Scene is still: "

  userPrmpt['content'] += current_scene + separator


  recipients = entry['Recipients']
  recipients_str = ""
  if len(recipients) == 0:
    recipients_str += "to themselves"
  elif len(recipients) == 1:
    recipients_str += recipients[0]
  else:
    recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

  userPrmpt['content'] += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."

  # append userPrmpt
  messages.append(userPrmpt)

  asstPrmpt['content'] += "Can you provide some context to help me with this decision?"
  # append asstPrmpt
  messages.append(asstPrmpt)

  userPrmpt2['content'] += "Yes, use the following as context:" + separator*2

  if entry['Scene'] != current_scene:
    current_scene = entry["Scene"]
    userPrmpt2['content'] += "The scene changes to: "
  else:
    userPrmpt2['content'] += "The Scene is still: "

  userPrmpt2['content'] += current_scene + separator

  for entry in context:
    if current_scene != entry['Scene']:
      current_scene = entry["Scene"]
      userPrmpt2['content'] += "The scene changes to: " + current_scene + separator

    recipients = entry['Recipients']
    recipients_str = ""
    if len(recipients) == 0:
      recipients_str += "to themselves"
    elif len(recipients) == 1:
      recipients_str += recipients[0]
    else:
      recipients_str += ", ".join(recipients[:-1]) + " and " + recipients[-1]

    userPrmpt2['content'] += f"{entry['Speaker']} says {quotation_token}{entry['Dialogue']}{quotation_token} to {recipients_str}."
    userPrmpt2['content'] += separator

  # append userPrmpt2
  messages.append(userPrmpt2)

  # userPrmpt2['content'] += separator + "-" *20 + separator*2

  # systemPrmpt2['content'] += "Using the context provided, return '1' if the target_moment was humorous or '0' if the target_moment was not humorous?"

  # append systemPrmpt2
  # messages.append(systemPrmpt2)

  # messages = [systemPrmpt, userPrmpt, asstPrmpt, ]

  return messages

### Sample Raw and Cleaned Formats:

In [6]:
filePath = "/content/info4940-sitcom/cleaned-data/S1/The Big Bang_S0101.json"
nSentenceContext = 5

with open(filePath, 'r') as file:
  data = json.load(file)
  data = [info for (_, info) in data.items()]

print(f"Example raw entry:")
display(data[3])

print(f"\n\nExample formatted entry ({nSentenceContext}context):\n")
input = transformInput(data[:3], data[3])

for i in input:
  print(f"Role: {i['role']} | Content:\n{i['content']}\n")

print(f'Actual: {("isHumor" in data[3])}')

Example raw entry:


{'EpisodeID': 'The Big Bang_S0101',
 'Scene': 'A corridor at a sperm bank.',
 'Recipients': ['Sheldon', 'Receptionist'],
 'Speaker': 'Leonard',
 'Dialogue': 'Excuse me.',
 'Dialogue Start Time': '00:00:23:140000',
 'Dialogue End Time': '00:00:25:060000'}



Example formatted entry (5context):

Role: system | Content:
You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0.

Role: user | Content:
Is the following target_moment from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond '1' for humorous or '0' for not humorous:

The Scene is still: A corridor at a sperm bank.
Leonard says "Excuse me." to Sheldon and Receptionist.

Role: assistant | Content:
Can you provide some context to help me with this decision?

Role: user | Content:
Yes, use the following as context:

The Scene is still: A corridor at a sperm bank.
Sheldon says "So if a photon is directed through a plane with two slits in it and either slit is observed, it will not go through both slits. If it's unobserved, it will. However, if it's observed after it's left 

# Creating sample data

In [7]:
filePaths = []
for i in range(5):
  fPath = '/content/info4940-sitcom/cleaned-data/S' + str(i+1)
  filePaths += list_files(fPath)

# getting random number for random state for consistency?
rand_seed1 = random.randint(1,100) # 64 #
rand_seed2 = random.randint(1,100) # 94 #
rand_seed3 = random.randint(1,100) # 69 #

# print([rand_seed1,rand_seed2,rand_seed3])

# sampling 10 episodes
nSamples = 10
test_size = (len(filePaths) - nSamples) / len(filePaths)
samplePaths, _ = train_test_split(filePaths, test_size=test_size, random_state=rand_seed1)

# cleaning/transforming inputs
nContext = 10
humorousData, nonHumorousData = fPathList_TO_DialogueWithContext(samplePaths, nContext)
sample_inputs, sample_labels = sampleEven(humorousData=humorousData, nonHumorousData=nonHumorousData, sample_size_per_class=250, rand_seed=rand_seed2)

# shuffling, not necessary but helpful for watching in-progress results
rand_order = [i for i in range(len(sample_inputs))]
random.seed(rand_seed3)
random.shuffle(rand_order)

inputs = [sample_inputs[i] for i in rand_order]
labels = [sample_labels[i] for i in rand_order]

# The LLAMA 3 8B instruct model

In [8]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, token=userdata.get('HF_TOKEN'))
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    token=userdata.get('HF_TOKEN')
)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

# Testing

In [9]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


input = inputs[0]
label = labels[0]

input_ids = tokenizer.apply_chat_template(
  input,
  add_generation_prompt=True,
  return_tensors="pt"
).to(model.device)

outputs = model.generate(
  input_ids,
  max_new_tokens=50,
  eos_token_id=terminators,
  do_sample=True,
  temperature=0.3,
  top_p=0.9,
  num_return_sequences=1,
)

response = outputs[0][input_ids.shape[-1]:]
response = tokenizer.decode(response, skip_special_tokens=True)

print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


0


In [10]:
print(input)
print(f'expected: {label}, model: {response}')

[{'role': 'system', 'content': "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."}, {'role': 'user', 'content': 'Is the following target_moment from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond \'1\' for humorous or \'0\' for not humorous:\n\nThe Scene is still: The apartment.\nPenny says "Oh, I know. Bernadette told me. Sorry, Sheldon. I know that game meant a lot to you. What-- " to Sheldon, Howard, Leonard and Raj.'}, {'role': 'assistant', 'content': 'Can you provide some context to help me with this decision?'}, {'role': 'user', 'content': 'Yes, use the following as context:\n\nThe Scene is still: The apartment.\nHoward says "Whoever did this knew what they were doing. He got in and out of your account in under 15 minutes, transferred all your stuff, and d

In [11]:
print(input[3]['content'])

Yes, use the following as context:

The Scene is still: The apartment.
Howard says "Whoever did this knew what they were doing. He got in and out of your account in under 15 minutes, transferred all your stuff, and didn't leave a digital fingerprint." to Sheldon, Leonard and Raj.
Sheldon says "Oh! There isn't enough chamomile tea in the world to quell the rage in my heart." to Howard, Leonard and Raj.
Raj says "Hold on. I'm talking to an orc under the bridge in Thunder Bluff who says if we pay him, he'll help us track down your things." to Sheldon, Howard and Leonard.
Sheldon says "Can we trust him?" to Howard, Leonard and Raj.
Raj says "I should say so; he appears to be a member of the Nigerian royal family." to Sheldon, Howard and Leonard.
Howard says "Whoa! Somebody's auctioning off a jeweled ostrich bridle!" to Sheldon, Leonard and Raj.
Sheldon says "No. Glenn's was leather." to Howard, Leonard and Raj.
Sheldon says "He was a simple ostrich. Is! Is, I haven't given up hope." to How

In [13]:
results, resultsData, pred, report = testLLAMA(inputs, labels, printResultsEveryN=10)

true = [str(i) for i in labels]
report = classification_report(y_pred=pred, y_true=true, target_names=["Non-Humorous","Humorous"])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Test: 1 of  500
Success rate: 100.00%

Results summary: {'truePos': 0, 'trueNeg': 1, 'falsePos': 0, 'falseNeg': 0}



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene

Test: 11 of  500
Success rate: 54.55%

Results summary: {'truePos': 5, 'trueNeg': 1, 'falsePos': 5, 'falseNeg': 0}



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene

...

Test: 481 of  500
Success rate: 54.05%

Results summary: {'truePos': 224, 'trueNeg': 36, 'falsePos': 202, 'falseNeg': 19}



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene

Test: 491 of  500
Success rate: 53.77%

Results summary: {'truePos': 227, 'trueNeg': 37, 'falsePos': 208, 'falseNeg': 19}



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene

Final success rate: 53.80%


In [14]:
print(report)
print(results)

              precision    recall  f1-score   support

Non-Humorous       0.66      0.16      0.25       250
    Humorous       0.52      0.92      0.67       250

    accuracy                           0.54       500
   macro avg       0.59      0.54      0.46       500
weighted avg       0.59      0.54      0.46       500

{'truePos': 230, 'trueNeg': 39, 'falsePos': 211, 'falseNeg': 20}


# Some interesting outputs along the way:

## user: "**What do you know about the sitcom the Big Bang Theory?**"

A popular and iconic sitcom! The Big Bang Theory is a American sitcom that originally aired from 2007 to 2019. Created by Chuck Lorre and Bill Prady, the show is a comedy about a group of socially awkward and intellectually brilliant scientists and their neighbor, a waitress and aspiring actress.

The show revolves around the lives of five main characters:

1. Sheldon Cooper (played by Jim Parsons), a theoretical physicist with Asperger's syndrome and a strict adherence to routine.
2. Leonard Hofstadter (played by Johnny Galecki), an experimental physicist and Sheldon's roommate and best friend.
3. Howard Wolowitz (played by Simon Helberg), an aerospace engineer and fellow physicist who lives across the hall from Leonard and Sheldon.
4. Rajesh "Raj" Koothrappali (played by Kunal Nayyar), an astrophysicist who is unable to speak in the presence of women unless he drinks alcohol.
5. Penny (played by Kaley Cuoco), a waitress and aspiring actress who lives across the hall from Leonard and Sheldon.

Throughout the show's 12 seasons, the characters navigate their personal and professional lives, often finding themselves in humorous and absurd situations. The show explores themes such as friendship, love, and the [cut off]

-------------------

## Prompt:
system: "You are humor classification model that takes a given line (the target) along with several precending lines (the context), and determines whether the target line was humorous (return '1') or not humorous (return '0'). RESTRICTION: ONLY RESPOND 1 OR 0."

user: 'Is the following target_moment from the sitcom, the Big Bang Theory, humorous or not humorous? Only respond '1' for humorous or '0' for not humorous:

The Scene is still: The Cafeteria
Sheldon says "It's not what it looks like." to Leonard.'

assistant: 'Can you provide some context to help me with this decision?'

user: 'Yes, use the following as context:

The Scene is still: The Cafeteria
Priya says "My brother-- he's got a big crush on Bernadette." to Leonard.
Leonard says "You're moving back to India?" to Priya.
Leonard says "What's going on?" to Raj, Penny, Raj and Howard.
Penny says "It's-It's not what it looks like." to Raj, Leonard, Raj and Howard.Sheldon says "What does it look like?" to Raj, Leonard, Raj and Howard.


## Model's Response:

Based on the context, I would classify the target moment as humorous. The phrase "It's not what it looks like" is a common idiomatic expression, but in this scene, it's being used in a way that's reminiscent of a soap opera or a melodrama, which adds a layer of humor to the situation. The fact that Sheldon is asking "What does it look like?" also implies that he's being obtuse and unaware of the obvious, which is a characteristic that's often played for comedic effect in the show.
